In [1]:
import src.data.breathe_data as br
import src.data.helpers as dh
import src.models.helpers as mh
import src.inference.helpers as ih
import src.modelling_fef2575.cpt_and_plots as cpt_and_plots
import src.models.var_builders as var_builders
import src.models.graph_builders as graph_builders
from src.inference.inf_algs import apply_bayes_net_bp, apply_factor_graph_bp

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import src.models.cpts.helpers as cpth

In [ ]:
# With DE
AR = mh.VariableNode("Airway resistance (%)", 0, 90, 2, prior=None)
DE = mh.DiscreteVariableNode("Days elapsed", 1, 3, 1)

cpt = cpth.get_cpt([AR, AR, DE], "_shift_span_[-20;20]_joint_sampling_3_days_model")

AR_light = mh.VariableNode("Airway resistance (%)", 0, 90, 10, prior=None)

In [ ]:
# With S
AR = mh.VariableNode("Airway resistance (%)", 0, 90, 2, prior=None)
# S = mh.DiscreteVariableNode("AR change factor shape", 2, 10, 2)
# S = mh.DiscreteVariableNode("AR change factor shape", 1, 3, 1)
S = mh.DiscreteVariableNode("AR change factor shape", 1, 1, 1, prior=None)

cpt = cpth.get_cpt([AR, AR, S], "_shape_factor_Gmain0.2_Gtails10_w0.73")

AR_light = mh.VariableNode("Airway resistance (%)", 0, 90, 10, prior=None)

In [4]:
AR1_light = mh.VariableNode(
    "Airway resistance day 1 (%)", 0, 90, 10, prior={"type": "uniform"}
)
AR2_light = mh.VariableNode(
    "Airway resistance day 2 (%)", 0, 90, 10, prior={"type": "uniform"}
)

print(cpt.shape)
# light_cpt = np.zeros((AR1_light.card, AR2_light.card, DE.card))
light_cpt = np.zeros((AR1_light.card, AR2_light.card, S.card))
print(light_cpt.shape)

# Marginalise the CPT over the light resolution
for i in range(AR1_light.card):
    for j in range(AR2_light.card):
        # for k in range(DE.card):
        for k in range(S.card):
            light_cpt[i, j, k] = np.sum(
                cpt[i * 5 : (i + 1) * 5, j * 5 : (j + 1) * 5, k]
            )

# Normalise the CPT
light_cpt = light_cpt / np.sum(light_cpt, axis=(0))

(45, 45, 1)
(9, 9, 1)


In [18]:
fig, title = cpth.plot_2d_cpt(
    light_cpt[:, :, 0],
    AR_light,
    AR_light,
    height=600,
    vspace=0.003,
    invert=False,
    p_range=[0, 1],
    y_label_two_lines=False,
)

In [5]:
cpth.save_cpt([AR_light, AR_light, S], light_cpt, suffix="_shape_factor_Gmain0.2_Gtails10_w0.73")
# cpth.save_cpt([AR_light, AR_light, S], light_cpt)

In [19]:
fig.show()